# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 364, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 364 (delta 132), reused 50 (delta 26), pack-reused 172 (from 1)
Receiving objects: 100% (364/364), 49.95 MiB | 21.81 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_LOSO
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


import tensorflow as tf
import numpy as np
import random
import os

# Establecer semilla
seed = 42

# Semillas para módulos principales
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

2025-05-05 13:14:36.218010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746450876.402363      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746450876.461183      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
from gmrrnet_adhd.models.spatio_temporal import prepare_streams_4s

X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

## Preprocesamiento de los datos mencionado por la propuesta

| Variable   | Forma resultante | Cálculo exacto                                                                                                                                                                                                                |
| ---------- | ---------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **`freq`** | `(N, 20, 1)`     | - PSD con `welch(signal, fs=128, nperseg=512)`.<br>- Potencia media en **20 bandas log‑espaciadas** entre 1 Hz y 64 Hz.<br>- Promedio sobre canales → vector de 20.<br>- Se añade un eje final de tamaño 1.                   |
| **`temp`** | `(N, 10, 1)`     | - Se recortan 510 muestras (de 512).<br>- Se dividen en **10 ventanas** consecutivas de 51 muestras (≈ 400 ms).<br>- **Media aritmética** dentro de cada ventana promediando canales.<br>- Se añade un eje final de tamaño 1. |
| **`spat`** | `(N, C, 1)`      | - Para cada canal: **RMS** del segmento `sqrt(mean(x**2))`.<br>- Se añade un eje final de tamaño 1.                                                                                                                           |


In [4]:
freq, temp, spat = prepare_streams_4s(X, fs=128)

freq.shape, temp.shape, spat.shape

((8213, 20, 1), (8213, 10, 1), (8213, 19, 1))

In [5]:
from sklearn.preprocessing import StandardScaler

scaler_f = StandardScaler().fit(freq.reshape(-1, 20))
scaler_t = StandardScaler().fit(temp.reshape(-1, 10))
scaler_s = StandardScaler().fit(spat.reshape(-1, spat.shape[1]))

freq = scaler_f.transform(freq.reshape(-1, 20)).reshape(freq.shape)
temp = scaler_t.transform(temp.reshape(-1, 10)).reshape(temp.shape)
spat = scaler_s.transform(spat.reshape(-1, spat.shape[1])).reshape(spat.shape)

# Importamos el modelo y definimos los hiperparámetros

In [6]:
from gmrrnet_adhd.models.spatio_temporal import build_eeg_attention_model
from tensorflow.keras.optimizers import Adam

model_name = "spatio_temporal"

model_args =    {'freq_shape' : freq.shape[1:],   # (20,1)
                 'temp_shape' : temp.shape[1:],   # (10,1)
                 'spat_shape' : spat.shape[1:]}   # (19,1)

compile_args = {'optimizer':lambda: Adam(1e-4, clipnorm=1.0),
    "loss": "categorical_crossentropy",
    "metrics": ["accuracy"]
}

model = build_eeg_attention_model(
    **model_args
)

model.summary()

I0000 00:00:1746450982.329852      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746450982.330518      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "EEG_Attention_Transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ freq_input (InputLayer)   │ (None, 20, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ temp_input (InputLayer)   │ (None, 10, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spat_input (InputLayer)   │ (None, 19, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 20, 1)          │              0 │ freq_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_2 (Cast)             │ (None, 10, 1)          │              0 │ temp_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_4 (Cast)             │ (None, 19, 1)          │              0 │ spat_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 20, 64)         │            128 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 10, 64)         │            128 │ cast_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 19, 64)         │            128 │ cast_4[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding       │ (None, 20, 64)         │              0 │ dense[0][0]            │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding_1     │ (None, 10, 64)         │              0 │ dense_6[0][0]          │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding_2     │ (None, 19, 64)         │              0 │ dense_12[0][0]         │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_1 (Cast)             │ (None, 20, 64)         │              0 │ positional_encoding[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_3 (Cast)             │ (None, 10, 64)         │              0 │ positional_encoding_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_5 (Cast)             │ (None, 19, 64)         │              0 │ positional_encoding_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_block │ (None, 20, 64)         │         83,200 │ cast_1[0][0]           │
│ (TransformerEncoderBlock) │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_bloc… │ (None, 10, 64)         │         83,200 │ cast_3[0][0]           │
│ (TransformerEncoderBlock) │                        │                │                        │
├──────────────────────

 Total params: 574,082 (2.19 MB)

 Trainable params: 574,082 (2.19 MB)

 Non-trainable params: 0 (0.00 B)

# Resultados - Leave One Subject Out

In [7]:
# X_total is a list: [freq, temp, spat]
X_total = [freq, temp, spat]            # each array shape (N, …)

results = train_LOSO(build_eeg_attention_model, X_total, y, sbjs, model_args, compile_args, 91, 120, model_name)

Evaluando modelo para el sujeto #91: v33p


I0000 00:00:1746451015.933500      72 service.cc:148] XLA service 0x7c12dc003640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746451015.934273      72 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746451015.934292      72 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746451020.263679      72 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746451037.346402      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step
Métricas para v33p: {'accuracy': 0.8672566371681416}

Evaluando modelo para el sujeto #92: v34p
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Métricas para v34p: {'accuracy': 0.8533333333333334}

Evaluando modelo para el sujeto #93: v35p
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step
Métricas para v35p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #94: v37p
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step
Métricas para v37p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #95: v38p
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Métricas para v38p: {'accuracy': 0.9789473684210527}

Evaluando modelo para el sujeto #96: v39p
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step
Métricas para v39p: {'accuracy': 0.9857142857142858}

Evaluando modelo para el sujeto #97: v3p
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 695ms/step
Métricas para v3p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #98: v40p
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step
Métricas para v40p: {'accuracy': 0.7662337662337663}

Evaluando modelo para el sujet

In [8]:
import pickle

with open(f'results_LOSO_{model_name}_2.pkl', 'wb') as f:
    pickle.dump(results, f)